In [1]:
# importing all necessary libraries to run the code
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import NaiveBayesClassifier
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
import re
# using the variable sw to hold all stopwords that are in English
sw = stopwords.words('english')

In [12]:
df = pd.read_csv("train_preprocess.tsv.txt",sep='\t')

In [14]:
df.head()

,text,respon
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    11000 non-null  object
 1   respon  11000 non-null  object
dtypes: object(2)
memory usage: 172.0+ KB


In [16]:
def cleaning_data(data):
    aux_list = []
    flag = False
    for phase_word in data:
        word_list = []
        for word in phase_word.split():
            word = word.lower()
            if flag and not word in sw:
                flag = False
                word_list.append('not_'+word)
                continue
            if re.search('(n\'t)$|(not)|(no)|(never)', word):
                flag = True
                continue
            if not word in sw:
                word = re.sub('[\W_0-9]', ' ', word)
                word_list.append(word)
        aux_list.append(' '.join(word_list))
    return aux_list

In [20]:
X = cleaning_data(df['text'])
y = df['respon']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [21]:
# This CountVectorizer is used to represent the words as a list of values, instead of text
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

vectorizer.fit(X)
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

In [24]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=len(vectorizer.get_feature_names())))
model.add(Dense(units=3, activation='sigmoid'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, 
          epochs=10, verbose=1)
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy:", scores[1])

Epoch 1/10
275/275 [==============================] - 7s 23ms/step - loss: 0.4637 - accuracy: 0.8138
Epoch 2/10
275/275 [==============================] - 6s 22ms/step - loss: 0.1884 - accuracy: 0.9369
Epoch 3/10
275/275 [==============================] - 8s 28ms/step - loss: 0.0911 - accuracy: 0.9701
Epoch 4/10
275/275 [==============================] - 7s 25ms/step - loss: 0.0533 - accuracy: 0.9817
Epoch 5/10
275/275 [==============================] - 6s 22ms/step - loss: 0.0354 - accuracy: 0.9877
Epoch 6/10
275/275 [==============================] - 6s 22ms/step - loss: 0.0261 - accuracy: 0.9885
Epoch 7/10
275/275 [==============================] - 6s 22ms/step - loss: 0.0204 - accuracy: 0.9926
Epoch 8/10
275/275 [==============================] - 6s 22ms/step - loss: 0.0166 - accuracy: 0.9952
Epoch 9/10
275/275 [==============================] - 6s 22ms/step - loss: 0.0136 - accuracy: 0.9973
Epoch 10/10
69/69 [==============================] - 0s 2ms/step - loss: 0.6795 - accuracy:

In [30]:
#save the model
model.save('mymodel.h5')
print("Model has created!")

Model has created!


In [31]:
# model evaluation
from keras.models import load_model

model = load_model('mymodel.h5')
scores = model.evaluate(X_test, y_test)

LSTM_accuracy = scores[1]*100

print('Test accuracy: ', scores[1]*100, '%')

69/69 [==============================] - 0s 2ms/step - loss: 0.6795 - accuracy: 0.8536
Test accuracy:  85.3636384010315 %


In [32]:
#A function for predict the sentiment
import re 
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

input_text = """makanan di sini tidak enak"""
input_text2 = """makanan di sini gak enak"""
input_text3 = """makanan di sini engga enak"""
input_text4 = """makanan di sini kurang enak"""
input_text5 = """presiden indonesia keren sekali"""
input_text6 = """presiden indonesia pergi ke timur tengah"""

def cleansing(sent):
    # Mengubah kata menjadi huruf kecil semua dengan menggunakan fungsi lower()
    string = sent.lower()
    # Menghapus emoticon dan tanda baca menggunakan "RegEx" dengan script di bawah
    string = re.sub(r'[^a-zA-Z0-9]', ' ', string)
    return string

sentiment = ['negative', 'neutral', 'positive']

text = [cleansing(input_text5)]
predicted = tokenizer.texts_to_sequences(text)
guess = pad_sequences(predicted, maxlen=X.shape[1])

model = load_model('mymodel.h5')
prediction = model.predict(guess)
polarity = np.argmax(prediction[0])
hasil = sentiment[polarity]

print("Text: %s" % text[0])
print("Sentiment: %s" % sentiment[polarity])

NameError: name 'tokenizer' is not defined